In [1]:
import pandas as pd
import altair as alt
import numpy as np
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
%run MakeNBImportAvailable.ipynb

In [3]:
import logging

In [4]:
log_format = "%(asctime)s::%(levelname)s::%(name)s::"\
             "%(filename)s::%(lineno)d::%(message)s"
logging.basicConfig(filename='Goalzero.log', level='DEBUG', format=log_format)

In [5]:
# https://www.belastingdienst.nl/wps/wcm/connect/bldcontentnl/belastingdienst/zakelijk/overige_belastingen/belastingen_op_milieugrondslag/tarieven_milieubelastingen/tabellen_tarieven_milieubelastingen?projectid=6750bae7%2D383b%2D4c97%2Dbc7a%2D802790bd1110
# bedrage ex btw
# S is Saldering
# Ook over ODE en Energiebelasting BTW heffing.
beleid = {'BTW_EH':0.21,'BTW_EL':0.21,'BTW_G':0.21,
          'EB_EH' : 0.03679 , 'EB_EL':0.03679 , 'EB_G':0.36322,
          'BT_EH' : 681.63/1.21/2 ,'BT_EL' :681.63/1.21/2 , 'BT_G' : 0,     
          'S_EH': 1, 'S_EL' : 1, 'S_G':1,
          'ODE_EH' : 0.0305 ,'ODE_EL' : 0.0305  ,'ODE_G' : 0.0865
         }

beleid = pd.DataFrame(data=beleid,index = ['1_2022_orig'])
beleid = beleid.transpose()

Teruggave Energiebelasting is gekoppeld aan electriciteitsaansluiting.   
Als geen verbruik, toch een terugggave

In [6]:
# verlaging belasting & verhoging korting
beleid['2_2022_aangepast'] = beleid['1_2022_orig'] 
beleid.loc['BTW_EH','2_2022_aangepast'] = 0.09
beleid.loc['BTW_EL','2_2022_aangepast'] = 0.09
beleid.loc['BTW_G','2_2022_aangepast'] = 0.09
beleid.loc['BT_EH','2_2022_aangepast'] = (681.63+265)/1.21/2
beleid.loc['BT_EL','2_2022_aangepast'] = (681.63+265)/1.21/2

beleid['3_2023'] = beleid['1_2022_orig'] 
beleid.loc['BTW_EH','3_2023'] = 0.21
beleid.loc['BTW_EL','3_2023'] = 0.21
beleid.loc['BTW_G','3_2023'] = 0.21
beleid.loc['BT_EH','3_2023'] = (681.63+265)/1.21/2
beleid.loc['BT_EL','3_2023'] = (681.63+265)/1.21/2


logging.info(beleid)
beleid

,1_2022_orig,2_2022_aangepast,3_2023
BTW_EH,0.210000,0.090000,0.210000
BTW_EL,0.210000,0.090000,0.210000
BTW_G,0.210000,0.090000,0.210000
EB_EH,0.036790,0.036790,0.036790
EB_EL,0.036790,0.036790,0.036790
EB_G,0.363220,0.363220,0.363220
BT_EH,281.665289,391.169421,391.169421
BT_EL,281.665289,391.169421,391.169421
BT_G,0.000000,0.000000,0.000000
S_EH,1.000000,1.000000,1.000000


In [7]:
prijzen_input = {'EH_p_kWh':0.245 ,'EL_p_kWh':0.195, 'G_p_m3':1.0765,
                 'net_EH_p_jr' : 267/2, 'net_EL_p_jr' : 267/2 , 'net_G_p_jr':186.55,
                 'vast_EH_p_jr' : 80/2 , 'vast_EL_p_jr' : 80/2 , 'vast_G_p_jr' : 80,
                 'TH_p_kWh' : 0.09 , 'TL_p_kWh' : 0.09,
                 'BTW_ref':0.21,
                 'refyr' : 2022}
prijzen_input = pd.DataFrame(prijzen_input,index=['2022_1'])
prijzen_input = prijzen_input.transpose()


prijsinput = pd.Series({'EH_p_kWh':0.41324 ,'EL_p_kWh':0.3285, 'G_p_m3':1.62,
                 'net_EH_p_jr' : 267/2, 'net_EL_p_jr' : 267/2 , 'net_G_p_jr':186.55,
                 'vast_EH_p_jr' : 80/2 , 'vast_EL_p_jr' : 80/2 , 'vast_G_p_jr' : 80,
                 'TH_p_kWh' : 0.09 , 'TL_p_kWh' : 0.09,
                 'BTW_ref':0.09,
                 'refyr' : 2022})
prijzen_input = pd.concat([prijzen_input,prijsinput],axis=1)
prijzen_input.rename(columns = {0 : '2022_2'},inplace=True)



prijsinput = pd.Series({'EH_p_kWh':0.55442 ,'EL_p_kWh':0.3285, 'G_p_m3':1.62,
                 'net_EH_p_jr' : 267/2, 'net_EL_p_jr' : 267/2 , 'net_G_p_jr':186.55,
                 'vast_EH_p_jr' : 80/2 , 'vast_EL_p_jr' : 80/2 , 'vast_G_p_jr' : 80,
                 'TH_p_kWh' : 0.0 , 'TL_p_kWh' : 0.0,
                 'BTW_ref':0.21,
                 'refyr' : 2022})

prijzen_input = pd.concat([prijzen_input,prijsinput],axis=1)
prijzen_input.rename(columns = {0 : '2022_3'},inplace=True)


prijsinput = pd.Series({'EH_p_kWh':0.245 ,'EL_p_kWh':0.195, 'G_p_m3':1.0765,
                 'net_EH_p_jr' : 267/2, 'net_EL_p_jr' : 267/2 , 'net_G_p_jr':186.55,
                 'vast_EH_p_jr' : 80/2 , 'vast_EL_p_jr' : 80/2 , 'vast_G_p_jr' : 80,
                 'TH_p_kWh' : 0.00 , 'TL_p_kWh' : 0.00,
                 'BTW_ref':0.21,
                 'refyr' : 2022})
prijzen_input = pd.concat([prijzen_input,prijsinput],axis=1)
prijzen_input.rename(columns = {0 : '2022_4'},inplace=True)



prijsinput = pd.Series({'EH_p_kWh':0.245 ,'EL_p_kWh':0.195, 'G_p_m3':1.0765,
                 'net_EH_p_jr' : 267/2, 'net_EL_p_jr' : 267/2 , 'net_G_p_jr':186.55,
                 'vast_EH_p_jr' : 80/2 , 'vast_EL_p_jr' : 80/2 , 'vast_G_p_jr' : 80,
                 'TH_p_kWh' : 0.05 , 'TL_p_kWh' : 0.05,
                 'BTW_ref':0.21,
                 'refyr' : 2022})
prijzen_input = pd.concat([prijzen_input,prijsinput],axis=1)
prijzen_input.rename(columns = {0 : '2022_5'},inplace=True)

prijsinput = pd.Series({'EH_p_kWh':0.4 ,'EL_p_kWh':0.4, 'G_p_m3':1.45,
                 'net_EH_p_jr' : 267/2, 'net_EL_p_jr' : 267/2 , 'net_G_p_jr':186.55,
                 'vast_EH_p_jr' : 80/2 , 'vast_EL_p_jr' : 80/2 , 'vast_G_p_jr' : 80,
                 'TH_p_kWh' : 0.12 , 'TL_p_kWh' : 0.12,
                 'BTW_ref':0.21,
                 'refyr' : 2022})
prijzen_input = pd.concat([prijzen_input,prijsinput],axis=1)
prijzen_input.rename(columns = {0 : '2023_plafond'},inplace=True)

prijsinput = pd.Series({'EH_p_kWh':0.73495 ,'EL_p_kWh':0.58801, 'G_p_m3':2.57841,
                 'net_EH_p_jr' : 267/2, 'net_EL_p_jr' : 267/2 , 'net_G_p_jr':186.55,
                 'vast_EH_p_jr' : 80/2 , 'vast_EL_p_jr' : 80/2 , 'vast_G_p_jr' : 80,
                 'TH_p_kWh' : 0.12 , 'TL_p_kWh' : 0.12,
                 'BTW_ref':0.21,
                 'refyr' : 2022})
prijzen_input = pd.concat([prijzen_input,prijsinput],axis=1)
prijzen_input.rename(columns = {0 : '2023_geenplafond'},inplace=True)


prijsinput = pd.Series({'EH_p_kWh':0.4 ,'EL_p_kWh':0.4, 'G_p_m3':1.45,
                 'net_EH_p_jr' : 267/2, 'net_EL_p_jr' : 267/2 , 'net_G_p_jr':186.55,
                 'vast_EH_p_jr' : 80/2 , 'vast_EL_p_jr' : 80/2 , 'vast_G_p_jr' : 80,
                 'TH_p_kWh' : 0.0 , 'TL_p_kWh' : 0.0,
                 'BTW_ref':0.21,
                 'refyr' : 2022})
prijzen_input = pd.concat([prijzen_input,prijsinput],axis=1)
prijzen_input.rename(columns = {0 : '2023_p_gT'},inplace=True)

prijsinput = pd.Series({'EH_p_kWh':0.73495 ,'EL_p_kWh':0.58801, 'G_p_m3':2.57841,
                 'net_EH_p_jr' : 267/2, 'net_EL_p_jr' : 267/2 , 'net_G_p_jr':186.55,
                 'vast_EH_p_jr' : 80/2 , 'vast_EL_p_jr' : 80/2 , 'vast_G_p_jr' : 80,
                 'TH_p_kWh' : 0.0 , 'TL_p_kWh' : 0.0,
                 'BTW_ref':0.21,
                 'refyr' : 2022})
prijzen_input = pd.concat([prijzen_input,prijsinput],axis=1)
prijzen_input.rename(columns = {0 : '2023_gp_gT'},inplace=True)


logging.info('prijzen: %s',prijzen_input)
prijzen_input

,2022_1,2022_2,2022_3,2022_4,2022_5,2023_plafond,2023_geenplafond,2023_p_gT,2023_gp_gT
EH_p_kWh,0.2450,0.41324,0.55442,0.2450,0.2450,0.40,0.73495,0.40,0.73495
EL_p_kWh,0.1950,0.32850,0.32850,0.1950,0.1950,0.40,0.58801,0.40,0.58801
G_p_m3,1.0765,1.62000,1.62000,1.0765,1.0765,1.45,2.57841,1.45,2.57841
net_EH_p_jr,133.5000,133.50000,133.50000,133.5000,133.5000,133.50,133.50000,133.50,133.50000
net_EL_p_jr,133.5000,133.50000,133.50000,133.5000,133.5000,133.50,133.50000,133.50,133.50000
net_G_p_jr,186.5500,186.55000,186.55000,186.5500,186.5500,186.55,186.55000,186.55,186.55000
vast_EH_p_jr,40.0000,40.00000,40.00000,40.0000,40.0000,40.00,40.00000,40.00,40.00000
vast_EL_p_jr,40.0000,40.00000,40.00000,40.0000,40.0000,40.00,40.00000,40.00,40.00000
vast_G_p_jr,80.0000,80.00000,80.00000,80.0000,80.0000,80.00,80.00000,80.00,80.00000
TH_p_kWh,0.0900,0.09000,0.00000,0.0000,0.0500,0.12,0.12000,0.00,0.00000


In [8]:
# # EH, EL : electriciteit aan net onttrokken, hoog/laag tarief [kWh]
# # G_SWW : Gasverbruik voor warmwater [m3]
# # G_VW : Gas voor verwarming [m3]
# # TH, TL: Terugleveren [kWH], hoog/laag tarief

# verbruik  = {'EH' : 2400, 'EL' :1600 , 'G_SWW':240 ,'G_VW':1180, 'TH' : 0 ,'TL':0}
# verbruik  = pd.DataFrame(verbruik, index=['2022'])
# verbruik = verbruik.transpose()
# verbruik
                         

In [9]:
def set_prijs_input(serie,beleid):
    '''
    input : is incl btw
    output : prijs voor berekening gaat zonder btw
    '''
    
    logging.info('set_prijs_input van inclusief naar exclusief btw')
    logging.info(serie)
    
    EH = serie['EH_p_kWh']/(1+serie['BTW_ref']) - beleid['ODE_EH'] - beleid['EB_EH']
    EL = serie['EL_p_kWh']/(1+serie['BTW_ref']) - beleid['ODE_EL'] - beleid['EB_EL']
    G = serie['G_p_m3']/(1+serie['BTW_ref']) - beleid['ODE_G'] - beleid['EB_G']
    net_EH = serie['net_EH_p_jr']/(1+serie['BTW_ref'])
    net_EL = serie['net_EL_p_jr']/(1+serie['BTW_ref'])
    net_G = serie['net_G_p_jr']/(1+serie['BTW_ref'])
    vast_EH = serie['vast_EH_p_jr']/(1+serie['BTW_ref'])
    vast_EL = serie['vast_EL_p_jr']/(1+serie['BTW_ref'])
    vast_G = serie['vast_G_p_jr']/(1+serie['BTW_ref'])
    TH = serie['TH_p_kWh']/(1+serie['BTW_ref'])
    TL = serie['TL_p_kWh']/(1+serie['BTW_ref'])

  
    
    result = pd.Series({'EH_p_kWh':EH ,'EL_p_kWh':EL, 'G_p_m3':G,
                 'net_EH_p_jr' : net_EH, 'net_EL_p_jr' : net_EL , 'net_G_p_jr': net_G,
                 'vast_EH_p_jr' : vast_EH , 'vast_EL_p_jr' : vast_EL , 'vast_G_p_jr' : vast_G,
                 'TH_p_kWh' : TH , 'TL_p_kWh' : TL,
                 'BTW_ref': serie['BTW_ref'],
                 'refyr' : serie['refyr']})
    
    logging.info('prijs exclusief btw en heffingen: ')
    logging.info(result)
    
    
    return result
    

In [10]:
set_prijs_input(prijzen_input['2023_geenplafond'],beleid['3_2023'])

EH_p_kWh           0.540107
EL_p_kWh           0.418669
G_p_m3             1.681197
net_EH_p_jr      110.330579
net_EL_p_jr      110.330579
net_G_p_jr       154.173554
vast_EH_p_jr      33.057851
vast_EL_p_jr      33.057851
vast_G_p_jr       66.115702
TH_p_kWh           0.099174
TL_p_kWh           0.099174
BTW_ref            0.210000
refyr           2022.000000
dtype: float64

In [11]:
set_prijs_input(prijzen_input['2022_3'],beleid['2_2022_aangepast'])

EH_p_kWh           0.390908
EL_p_kWh           0.204198
G_p_m3             0.889123
net_EH_p_jr      110.330579
net_EL_p_jr      110.330579
net_G_p_jr       154.173554
vast_EH_p_jr      33.057851
vast_EL_p_jr      33.057851
vast_G_p_jr       66.115702
TH_p_kWh           0.000000
TL_p_kWh           0.000000
BTW_ref            0.210000
refyr           2022.000000
dtype: float64

In [12]:
beleid

,1_2022_orig,2_2022_aangepast,3_2023
BTW_EH,0.210000,0.090000,0.210000
BTW_EL,0.210000,0.090000,0.210000
BTW_G,0.210000,0.090000,0.210000
EB_EH,0.036790,0.036790,0.036790
EB_EL,0.036790,0.036790,0.036790
EB_G,0.363220,0.363220,0.363220
BT_EH,281.665289,391.169421,391.169421
BT_EL,281.665289,391.169421,391.169421
BT_G,0.000000,0.000000,0.000000
S_EH,1.000000,1.000000,1.000000


# Verbruik en teruglevering


In [13]:
Gas = {'VW_m3_p_jr' : 800 , 'SWW_m3_p_jr' :300}
Gas = pd.DataFrame(Gas,index=['huidig'])
Gas = Gas.transpose()
Gas

,huidig
VW_m3_p_jr,800
SWW_m3_p_jr,300


In [14]:
E_gem_jaar =  pd.read_pickle(".//E_gem_perjaar.pkl") 
E_gem_jaar

,yr,Elektriciteit_cumul,opbrengst_cumul,gebruik_eigen_cumul,gebruik_net_cumul,terugleveren_cumul,zon_laag_kWh_cumul,zon_hoog_kWh_cumul,eigengebruik_laag_kWh_cumul,eigengebruik_hoog_kWh_cumul,gebruiknet_laag_kWh_cumul,gebruiknet_hoog_kWh_cumul,teruglever_laag_kWh_cumul,teruglever_hoog_kWh_cumul
Wp,,,,,,,,,,,,,,
0,2016.0,4107.069909,0.000000,0.000000,4107.069909,0.000000,0.000000,0.000000,0.000000,0.000000,1552.184545,2554.885364,0.000000,0.000000
1000,2016.0,4107.069909,880.559793,753.867654,3353.202255,126.692139,287.606051,592.953742,249.465807,504.401847,1302.718739,2050.483516,38.140244,88.551895
2000,2016.0,4107.069909,1761.119586,1120.345545,2986.724364,640.774040,575.212101,1185.907485,369.570520,750.775025,1182.614025,1804.110338,205.641581,435.132460
3000,2016.0,4107.069909,2641.679379,1323.979280,2783.090629,1317.700098,862.818152,1778.861227,435.546413,888.432867,1116.638133,1666.452496,427.271739,890.428360
4000,2016.0,4107.069909,3522.239172,1458.187076,2648.882833,2064.052096,1150.424202,2371.814970,478.890318,979.296758,1073.294227,1575.588606,671.533884,1392.518212
5000,2016.0,4107.069909,4402.798965,1555.956946,2551.112963,2846.842019,1438.030253,2964.768712,510.375540,1045.581405,1041.809005,1509.303958,927.654712,1919.187307
6000,2016.0,4107.069909,5283.358758,1631.750470,2475.319439,3651.608288,1725.636303,3557.722455,534.795614,1096.954856,1017.388932,1457.930508,1190.840689,2460.767598
7000,2016.0,4107.069909,6163.918551,1692.768239,2414.301670,4471.150312,2013.242354,4150.676197,554.545354,1138.222885,997.639192,1416.662479,1458.697000,3012.453312
8000,2016.0,4107.069909,7044.478343,1743.498838,2363.571071,5300.979505,2300.848404,4743.629939,571.125697,1172.373141,981.058848,1382.512222,1729.722707,3571.256798


In [15]:
colsout=['yr']
# E_gem_jaar.loc[:, ~E_gem_jaar.columns.isin(colsout)]
source = pd.melt(E_gem_jaar.loc[:, ~E_gem_jaar.columns.isin(colsout)].reset_index(),id_vars=['Wp'])

In [16]:
chart = alt.Chart(source).mark_bar(clip=True,width=15).encode(
        x=alt.X('Wp:Q'),
        y=alt.Y('value:Q'),
        color=alt.Color('variable:N', scale=alt.Scale(
            scheme='dark2'))).properties(width=200,height=200).interactive().facet('variable:N',columns=3)
    
display(chart)



alt.FacetChart(...)

In [17]:
E_gem_jaar.columns

Index(['yr', 'Elektriciteit_cumul', 'opbrengst_cumul', 'gebruik_eigen_cumul',
       'gebruik_net_cumul', 'terugleveren_cumul', 'zon_laag_kWh_cumul',
       'zon_hoog_kWh_cumul', 'eigengebruik_laag_kWh_cumul',
       'eigengebruik_hoog_kWh_cumul', 'gebruiknet_laag_kWh_cumul',
       'gebruiknet_hoog_kWh_cumul', 'teruglever_laag_kWh_cumul',
       'teruglever_hoog_kWh_cumul'],
      dtype='object')

In [18]:
colsforcalc = ['zon_laag_kWh_cumul','zon_hoog_kWh_cumul',
               'eigengebruik_laag_kWh_cumul','eigengebruik_hoog_kWh_cumul',
               'gebruiknet_laag_kWh_cumul','gebruiknet_hoog_kWh_cumul', 
               'teruglever_laag_kWh_cumul','teruglever_hoog_kWh_cumul']

In [19]:
E_gem_jaar[colsforcalc]

,zon_laag_kWh_cumul,zon_hoog_kWh_cumul,eigengebruik_laag_kWh_cumul,eigengebruik_hoog_kWh_cumul,gebruiknet_laag_kWh_cumul,gebruiknet_hoog_kWh_cumul,teruglever_laag_kWh_cumul,teruglever_hoog_kWh_cumul
Wp,,,,,,,,
0,0.000000,0.000000,0.000000,0.000000,1552.184545,2554.885364,0.000000,0.000000
1000,287.606051,592.953742,249.465807,504.401847,1302.718739,2050.483516,38.140244,88.551895
2000,575.212101,1185.907485,369.570520,750.775025,1182.614025,1804.110338,205.641581,435.132460
3000,862.818152,1778.861227,435.546413,888.432867,1116.638133,1666.452496,427.271739,890.428360
4000,1150.424202,2371.814970,478.890318,979.296758,1073.294227,1575.588606,671.533884,1392.518212
5000,1438.030253,2964.768712,510.375540,1045.581405,1041.809005,1509.303958,927.654712,1919.187307
6000,1725.636303,3557.722455,534.795614,1096.954856,1017.388932,1457.930508,1190.840689,2460.767598
7000,2013.242354,4150.676197,554.545354,1138.222885,997.639192,1416.662479,1458.697000,3012.453312
8000,2300.848404,4743.629939,571.125697,1172.373141,981.058848,1382.512222,1729.722707,3571.256798


In [20]:
# zon_laag + zon_hoog = eigengebruik_laag_kWh_cumul +eigengebruik_hoog_kWh_cumul +  	teruglever_laag_kWh_cumul 	teruglever_hoog_kWh_cumul

In [21]:
def checkjaarhoeveelheden(row):
    logging.info('function checkjaarhoeveelheden')
    zon = row['zon_laag_kWh_cumul']+row['zon_hoog_kWh_cumul']
    eigen = row['eigengebruik_laag_kWh_cumul']+row['eigengebruik_hoog_kWh_cumul']
    terug = row['teruglever_laag_kWh_cumul']+row['teruglever_hoog_kWh_cumul']
    uit_net = row['gebruiknet_laag_kWh_cumul']+row['gebruiknet_hoog_kWh_cumul']
    
    return pd.Series({'zon':zon,
                      'eigen':eigen,
                      'uit_net':uit_net,
                      'verbruik':eigen+uit_net,
                      'terug':terug,
                      'checkzon':zon - eigen - terug})
                     
    

In [22]:
# report de jaargegevens per categorie en check dat teruglevering + eigen gebruik gelijk is aan totale zonopbrengst
checkjaarhoeveelheden(E_gem_jaar.loc[2000])

zon         1.761120e+03
eigen       1.120346e+03
uit_net     2.986724e+03
verbruik    4.107070e+03
terug       6.407740e+02
checkzon    1.136868e-13
dtype: float64

In [23]:
# EH_p_kWh           0.311829
# EL_p_kWh           0.234086
# G_p_m3             1.036519
# net_EH_p_jr      122.477064
# net_EL_p_jr      122.477064
# net_G_p_jr       171.146789
# vast_EH_p_jr      36.697248
# vast_EL_p_jr      36.697248
# vast_G_p_jr       73.394495
# TH_p_kWh           0.082569
# TL_p_kWh           0.082569
# BTW_ref            0.090000
# refyr           2022.000000

In [24]:
def pkeerQ(var,p,Q,beleid,s):
    
#     var_var = ['ODE','EB','net','vast','BTW','BT']
#     var_var = [''.join([s,'_',var]) for s in var]
#     print(var_var)
    if 'm3' in var:
        logging.info('GAS :')
        
        prijs = p['G_p_m3']
        logging.info('GAS : %s. met prijs ex btw:%s',str(var),prijs)  
        kosten = prijs * Q * (1+beleid['BTW_G']) + Q*(beleid['ODE_G'] + beleid['EB_G'])
        logging.info('Verwarming, toevoegen Energiebelasting: %s en teruggave van belasting %s',beleid['EB_G'],beleid['BT_G'])
        logging.info('prijs * Q * (1+btw) + Q*(ODE + EB_G %s', kosten)
        
        if var=='VW_m3_p_jr':
            # voeg energie belasting toe en teruggave belasting  EN voeg vaste + net kosten 
            kosten= kosten + beleid['EB_G'] - beleid['BT_G']
            
            kosten = kosten + p['net_G_p_jr']*(1+beleid['BTW_G']) +p['vast_G_p_jr']*(1+beleid['BTW_G'])
        logging.info(kosten)
        
        opbrengst_terug = 0
        uitgespaard = 0
        
        
    else:
        
        logging.info('Electriciteit:')
        
        
        if var=='EH' :
            btw = beleid['BTW_EH']
            ODE = beleid['ODE_EH']
            Energiebelasting = beleid['EB_EH']
            belastingterug = beleid['BT_EH']
            
            prijs = p['EH_p_kWh']       
            net = p['net_EH_p_jr']
            vast = p['vast_EH_p_jr']
            prijs_terug = p['TH_p_kWh']
    
            logging.info('E : %s. met prijs ex btw:%s',str(var),prijs)  
        
            gebruik_net = Q.loc['gebruiknet_hoog_kWh_cumul']
            terug_lever = Q.loc['teruglever_hoog_kWh_cumul']
            eigengebruik = Q.loc['eigengebruik_hoog_kWh_cumul']
            
        else:
            btw = beleid['BTW_EL']
            ODE = beleid['ODE_EL']
            Energiebelasting = beleid['EB_EL']
            belastingterug = beleid['BT_EL']
            
            prijs = p['EL_p_kWh']
            net = p['net_EL_p_jr']
            vast = p['vast_EL_p_jr']
            prijs_terug = p['TL_p_kWh']
            
            logging.info('E : %s. met prijs ex btw:%s',str(var),prijs)  
            
            gebruik_net = Q.loc['gebruiknet_laag_kWh_cumul']
            terug_lever = Q.loc['teruglever_laag_kWh_cumul']
            eigengebruik = Q.loc['eigengebruik_laag_kWh_cumul']
            
        logging.info('gebruik net :%s \n terug_lever :%s \n eigengebruik %s',gebruik_net,terug_lever,eigengebruik)
        
        # 100 pct salderen dan voor terug leveren zelfde tarief als voor onttrekking van net.
        # als 20 pct salderen 
        # stel 5000 opwekken en stel gelijk aan totaal verbruik, 
        # waarvan 1000 eigen gebruik,  
        # dus 4000 opgenomen en op ander tijdstip teruggeleverd. 
        # 20 pct van 4000 =800 tegen zelfde lever tarief als 
        # rest tegen terug lever tarief (dus zonder heffingen etc.)
        
        kosten = gebruik_net * prijs * (1+btw) + (net+vast)*(1+btw) + \
                gebruik_net * (ODE + Energiebelasting) \
                - belastingterug
        logging.info('kosten: \n')
        
        logging.info('plus gebruik_net*prijs*(1+btw) = %s',gebruik_net * prijs * (1+btw))
        logging.info('plus vaste kosten netgebruik + vaste aansluiting * (1+btw) = %s',(net+vast)*(1+btw))
        logging.info('plus gebruik_net * (ODE + Energiebelasting) = %s',gebruik_net * (ODE + Energiebelasting))
        logging.info('minus belastingterug = %s',belastingterug)        
        logging.info('kosten : %s',kosten)
        
        uitgespaard = eigengebruik * prijs*(1+btw) + eigengebruik * (ODE + Energiebelasting)
        logging.info('uitgespaard : %s',uitgespaard)
        
        # impact van saldering :
        # mogelijk wordt er meer terug geleverd dan onttrokken aan het net, dan saldering max gelijk aan onttrokken hoeveelheid
        # en het overschrijdende deel tegen terug lever tarief \
        logging.info('kies minimum van gebruik_net of terug_lever')
        max_saldering_kwh=np.min([gebruik_net,terug_lever])
        logging.info('max salderingshoeveelheid : %s',max_saldering_kwh)
        

            # stel 4000 terug_lever, gebruik_net 1000
            # dan max saldering is 1000.
            # als gedeeltelijke saldering bijv 60 pct: 60 * 1000 = 600  dat tegen volledig onttrekkingstarief
            # dan saldering 600 is dus pct * min(terug_lever,gebruik_net)
            # rest 4000 - 600 tegen terug lever tarief.
            # terug_lever - max saldering is hoeveelheid tegen 
            # 
            # wat als 500 terug lever en gebruik net 10000
            # dan saldering is max 500
            # 
        
        logging.info('terug geleverd %s tegen levertarief prijs pct regeling %s',max_saldering_kwh,s)
        logging.info('als 100 pct saldering %s', max_saldering_kwh)
        gesaldeerd_kwh = s/100 * max_saldering_kwh
        logging.info('deze regeling :%s',gesaldeerd_kwh)
        
        gesaldeerd_eur = gesaldeerd_kwh * prijs * (1+btw) + gesaldeerd_kwh *(ODE+Energiebelasting)
        
     
        # niet gesaldeerde hoeveelheid + hoeveelheid dat meer dan gebruik_net is moet worden teruggeleverd tegen productie tarief
        restteruglever_kwh = terug_lever - gesaldeerd_kwh
        logging.info('terug geleverd %s tegen terug lever tarief %s', restteruglever_kwh,prijs_terug)  
        terugleveropbrengst = restteruglever_kwh * prijs_terug * (1+btw)
        logging.info('opbrengst teruglevering :%s', terugleveropbrengst)  
        
        opbrengst_terug =   gesaldeerd_eur + terugleveropbrengst
        
        logging.info('opbrengst_terug : %s',opbrengst_terug)
        logging.info('\n')
             

    
    return kosten, opbrengst_terug, uitgespaard
    # return kosten, opbrengst_terug, uitgespaard

In [25]:
def bereken_kosten(prijs_input,Elec,Gas,beleid,pct_saldering):
    
    '''
    input: 
        prijzen incl btw
        verbruik Electriciteit: uit_net hoog+laag , teruglever hoog+laag, eigen_hoog+laag
        verbruik Gas SWW, Gas verwarming
        percentage saldering
    
    berekening: 
        prijs naar ex btw en ex heffingen
        
    
    output: op jaarbasis incl btw en heffingen
        kosten electriciteit laag
        kosten electriciteit hoog
        kosten gas vw
        kosten gas sww
        
        opbrengst teruglever laag
        opbrengst teruglever hoog
        
        uitgespaarde kosten eigen gebruik laag
        uitgespaarde kosten eigen gebruik hoog
        
    '''
    # bereken de prijzen excl heffingen en btw
    prijs_ex = set_prijs_input(prijs_input,beleid)
    
    
    
    Gas_VW,_,_ =  pkeerQ('VW_m3_p_jr', prijs_ex, Gas.loc['VW_m3_p_jr'], beleid, pct_saldering)
    Gas_SWW,_,_ =  pkeerQ('SWW_m3_p_jr', prijs_ex, Gas.loc['SWW_m3_p_jr'], beleid, pct_saldering)
    EH, E_terug_H, E_eigen_H = pkeerQ('EH',prijs_ex,Elec,beleid,pct_saldering)
    EL, E_terug_L, E_eigen_L = pkeerQ('EL',prijs_ex,Elec,beleid,pct_saldering)

    logging.info('Gas verwarming %s',str(Gas_VW))
    logging.info('Gas SWW %s',str(Gas_SWW.squeeze()))
    logging.info('EH verbruik %s, EH terug %s, EH bespaard %s',EH, E_terug_H, E_eigen_H)
    logging.info('EL verbruik %s, EL terug %s, EL bespaard %s',EL, E_terug_L, E_eigen_L)
    logging.info('Totaal : %s',Gas_VW + Gas_SWW + EH - E_terug_H + EL - E_terug_L)
    return pd.Series({'EH':EH,
                     'EL':EL,
                     'Gas_VW':Gas_VW.squeeze(),
                     'Gas_SWW':Gas_SWW.squeeze(),
                     'E_terug_H':E_terug_H,
                     'E_terug_L':E_terug_L,
                     'E_eigen_H':E_eigen_H,
                     'E_eigen_L':E_eigen_L})
    
    
    

In [26]:
def toongetallen(prijs,verbruik_E,verbruik_G,beleid,salderin):
    
    a = bereken_kosten(prijs,verbruik_E,verbruik_G,beleid,salderin)
    print(a)
    
    E_kost = a['EH'] - a['E_terug_H'] + a['EL'] - a['E_terug_L']
    T_kost = E_kost+a['Gas_VW']+a['Gas_SWW']
    
    print('kosten per jaar :',round(T_kost,0))
    
    print('kosten per maand:',round(T_kost/12,1))
    
    return (a)

In [27]:
hihi=toongetallen(prijzen_input['2023_plafond'],E_gem_jaar.loc[8000],Gas,beleid['3_2023'],0)

EH            315.799325
EL            160.890873
Gas_VW       1351.360260
Gas_SWW       406.667640
E_terug_H     428.550816
E_terug_L     207.566725
E_eigen_H     452.382569
E_eigen_L     220.379759
dtype: float64
kosten per jaar : 1599.0
kosten per maand: 133.2


In [28]:
hihi['EH']


315.7993254402833

In [29]:
toongetallen(prijzen_input['2023_plafond'],E_gem_jaar.loc[0],Gas,beleid['3_2023'],100)

EH            768.181894
EL            381.270632
Gas_VW       1351.360260
Gas_SWW       406.667640
E_terug_H       0.000000
E_terug_L       0.000000
E_eigen_H       0.000000
E_eigen_L       0.000000
dtype: float64
kosten per jaar : 2907.0
kosten per maand: 242.3


EH            768.181894
EL            381.270632
Gas_VW       1351.360260
Gas_SWW       406.667640
E_terug_H       0.000000
E_terug_L       0.000000
E_eigen_H       0.000000
E_eigen_L       0.000000
dtype: float64

In [30]:
toongetallen(prijzen_input['2023_plafond'],E_gem_jaar.loc[2000],Gas,beleid['3_2023'],100)

EH            478.481011
EL            238.664788
Gas_VW       1351.360260
Gas_SWW       406.667640
E_terug_H     167.904171
E_terug_L      79.350732
E_eigen_H     289.700883
E_eigen_L     142.605844
dtype: float64
kosten per jaar : 2228.0
kosten per maand: 185.7


EH            478.481011
EL            238.664788
Gas_VW       1351.360260
Gas_SWW       406.667640
E_terug_H     167.904171
E_terug_L      79.350732
E_eigen_H     289.700883
E_eigen_L     142.605844
dtype: float64

In [31]:
toongetallen(prijzen_input['2023_geenplafond'],E_gem_jaar.loc[0],Gas,beleid['3_2023'],100)

EH           1623.940747
EL            673.096848
Gas_VW       2254.088260
Gas_SWW       745.190640
E_terug_H       0.000000
E_terug_L       0.000000
E_eigen_H       0.000000
E_eigen_L       0.000000
dtype: float64
kosten per jaar : 5296.0
kosten per maand: 441.4


EH           1623.940747
EL            673.096848
Gas_VW       2254.088260
Gas_SWW       745.190640
E_terug_H       0.000000
E_terug_L       0.000000
E_eigen_H       0.000000
E_eigen_L       0.000000
dtype: float64

In [32]:
# a['EH'] - a['E_terug_H'] + a['EL'] - a['E_terug_L']

In [33]:
toongetallen(prijzen_input['2022_2'],E_gem_jaar.loc[4000],Gas,beleid['2_2022_aangepast'],100)

EH            423.884892
EL            128.407755
Gas_VW       1530.533380
Gas_SWW       473.857560
E_terug_H     567.010996
E_terug_L     216.532004
E_eigen_H     398.753873
E_eigen_L     154.415262
dtype: float64
kosten per jaar : 1773.0
kosten per maand: 147.8


EH            423.884892
EL            128.407755
Gas_VW       1530.533380
Gas_SWW       473.857560
E_terug_H     567.010996
E_terug_L     216.532004
E_eigen_H     398.753873
E_eigen_L     154.415262
dtype: float64

In [34]:
# a['EH'] - a['E_terug_H'] + a['EL'] - a['E_terug_L']

In [35]:
a = bereken_kosten(prijzen_input['2022_2'],E_gem_jaar.loc[1000],Gas,beleid['2_2022_aangepast'],100)
a

EH            617.254454
EL            202.384289
Gas_VW       1530.533380
Gas_SWW       473.857560
E_terug_H      36.056906
E_terug_L      12.298089
E_eigen_H     205.384311
E_eigen_L      80.438728
dtype: float64

In [36]:
pd.DataFrame(a).transpose()

,EH,EL,Gas_VW,Gas_SWW,E_terug_H,E_terug_L,E_eigen_H,E_eigen_L
0,617.254454,202.384289,1530.53338,473.85756,36.056906,12.298089,205.384311,80.438728


In [37]:
a['EH'] - a['E_terug_H'] + a['EL'] - a['E_terug_L']

771.2837478509111

In [38]:

s_dummy = pd.DataFrame(range(2023,2060),columns=['yr'])
s_dummy['pct']=0
s_dummy.loc[s_dummy['yr']<2025,['pct']]=100
s_dummy.loc[(s_dummy['yr']>=2025)&(s_dummy['yr']<2027),['pct']]=64
s_dummy.loc[s_dummy['yr']==2027,['pct']]=55
s_dummy.loc[s_dummy['yr']==2028,['pct']]=44
s_dummy.loc[s_dummy['yr']==2029,['pct']]=37
s_dummy.loc[s_dummy['yr']==2030,['pct']]=28

# s_dummy[0:10]
s_dummy.set_index(['yr'],inplace=True)
# s_dummy = s_dummy.to_dict('dict')
# s_dummy.keys()
s_dummy.loc[2028].squeeze()

44

In [39]:
saldering_dict = { 2023:100, 2024:100,
                 2025 : 64, 2026:64,
                 2027 : 55, 2028:46,
                 2029 : 37, 2030:28,
                 2031 : 0,
                 2032:0,
                 2033:0,
                 2034:0,
                 2035:0,
                 2036:0
                 }

In [40]:
# TO DO Create saldering based on now called s_dummy

In [41]:
def berekenjaren(prijzen,E_verbruik,G_verbruik,beleid_loc,saldering,inv_per_1000Wp,Wp):
    '''berekening impact elk jaar '''
    
    df = []
    nryrs = 25
    pct = 100
    Wp_panelen = E_verbruik.name
    # print(m2panelen)
    for y in range(2023,2023+nryrs):
        a = bereken_kosten(prijzen,E_verbruik,G_verbruik,beleid_loc,saldering.loc[y].squeeze())
        df.append(a.transpose())
    
    df = pd.DataFrame(df)
    df['gas'] = df['Gas_VW']+df['Gas_SWW']
    df['el'] = df['EH'] + df['EL'] - df['E_terug_H'] - df['E_terug_L']
    df['gas_el'] = df['gas'] + df['el']
    df['gas_el_cumul'] = df['gas_el'].cumsum()
    df['investment'] = Wp * inv_per_1000Wp 
    df['total'] = df['gas_el_cumul'] + df['investment']
    df['invper1kWp']=inv_per_1000Wp
    df['Wp'] = Wp
    
    return df

In [42]:
# gemiddelde installatie kosten per m2
prijsrange = np.arange(1,2.5,0.250)

In [43]:
def berekenbesparing(prijsref,df_input):
    df = pd.DataFrame({})
    
    for p in prijsrange:
        for Wp in E_gem_jaar.index:
            a1 = prijsref-df_input[(df_input['Wp']==Wp)&(df_input['invper1kWp']==p)]['gas_el'] 
            # df = df.append({'beparing': a1}, ignore_index=True)
            df = pd.concat([df, a1])
        # df_besparing = df_besparing.append(a1,ignore_index=False,axis=1)
    df.rename({0 : 'besparing' },inplace=True,axis=1)
    return df

In [44]:
def berekencashflow(selectedbeleid,selectedprijs):
    
    df = pd.DataFrame()
    for p in prijsrange:
        for Wp in E_gem_jaar.index:
            a1 = berekenjaren(selectedprijs,E_gem_jaar.loc[Wp],Gas,selectedbeleid,s_dummy,p,Wp)
            df = df.append(a1,ignore_index=False)
    df['selectedbeleid'] = selectedbeleid.name
    df['selectedprijs']=selectedprijs.name
    
    # set de referentie om delta en rendement te kunnen berekenen
    prijsref = df[(df['invper1kWp']==1)&(df['Wp']==0)]['gas_el']
    df['besparing'] = berekenbesparing(prijsref,df)
    df['rendement'] = df['besparing'] / df['investment']
    
    return df

In [45]:
df_oud = berekencashflow(beleid['1_2022_orig'],prijzen_input['2022_1'])

In [46]:
df_huidig = berekencashflow(beleid['2_2022_aangepast'],prijzen_input['2022_2'])

In [47]:
df_volgend = berekencashflow(beleid['2_2022_aangepast'],prijzen_input['2022_3'])

In [48]:
# df_zonderterugleveropbrengst = berekencashflow(beleid['1_2022_orig'],prijzen_input['2022_4'])

In [49]:
df_2023geenplafond = berekencashflow(beleid['3_2023'],prijzen_input['2023_geenplafond'])

In [50]:
df_2023plafond = berekencashflow(beleid['3_2023'],prijzen_input['2023_plafond'])

In [51]:
df_2023_gp_geenT = berekencashflow(beleid['3_2023'],prijzen_input['2023_gp_gT'])

In [52]:
df_2023_p_geenT = berekencashflow(beleid['3_2023'],prijzen_input['2023_p_gT'])

In [53]:
df_2023geenplafond.head(10)

,EH,EL,Gas_VW,Gas_SWW,E_terug_H,E_terug_L,E_eigen_H,E_eigen_L,gas,el,gas_el,gas_el_cumul,investment,total,invper1kWp,Wp,selectedbeleid,selectedprijs,besparing,rendement
0,1623.940747,673.096848,2254.08826,745.19064,0.0,0.0,0.0,0.0,2999.2789,2297.037595,5296.316495,5296.316495,0.0,5296.316495,1.0,0,3_2023,2023_geenplafond,0.0,NaN
1,1623.940747,673.096848,2254.08826,745.19064,0.0,0.0,0.0,0.0,2999.2789,2297.037595,5296.316495,10592.632991,0.0,10592.632991,1.0,0,3_2023,2023_geenplafond,0.0,NaN
2,1623.940747,673.096848,2254.08826,745.19064,0.0,0.0,0.0,0.0,2999.2789,2297.037595,5296.316495,15888.949486,0.0,15888.949486,1.0,0,3_2023,2023_geenplafond,0.0,NaN
3,1623.940747,673.096848,2254.08826,745.19064,0.0,0.0,0.0,0.0,2999.2789,2297.037595,5296.316495,21185.265982,0.0,21185.265982,1.0,0,3_2023,2023_geenplafond,0.0,NaN
4,1623.940747,673.096848,2254.08826,745.19064,0.0,0.0,0.0,0.0,2999.2789,2297.037595,5296.316495,26481.582477,0.0,26481.582477,1.0,0,3_2023,2023_geenplafond,0.0,NaN
5,1623.940747,673.096848,2254.08826,745.19064,0.0,0.0,0.0,0.0,2999.2789,2297.037595,5296.316495,31777.898973,0.0,31777.898973,1.0,0,3_2023,2023_geenplafond,0.0,NaN
6,1623.940747,673.096848,2254.08826,745.19064,0.0,0.0,0.0,0.0,2999.2789,2297.037595,5296.316495,37074.215468,0.0,37074.215468,1.0,0,3_2023,2023_geenplafond,0.0,NaN
7,1623.940747,673.096848,2254.08826,745.19064,0.0,0.0,0.0,0.0,2999.2789,2297.037595,5296.316495,42370.531963,0.0,42370.531963,1.0,0,3_2023,2023_geenplafond,0.0,NaN
8,1623.940747,673.096848,2254.08826,745.19064,0.0,0.0,0.0,0.0,2999.2789,2297.037595,5296.316495,47666.848459,0.0,47666.848459,1.0,0,3_2023,2023_geenplafond,0.0,NaN
9,1623.940747,673.096848,2254.08826,745.19064,0.0,0.0,0.0,0.0,2999.2789,2297.037595,5296.316495,52963.164954,0.0,52963.164954,1.0,0,3_2023,2023_geenplafond,0.0,NaN


In [54]:
# plot cumulatieve kosten
def plot_cumul_costs(df):
    source = df.reset_index()
    # source = source[(source['index']<10)&(source['index']>3)]
    chart = alt.Chart(source).mark_line(clip=True,width=15).encode(
            x=alt.X('index:N'),
            y=alt.Y('total:Q'),
            color=alt.Color('Wp:N', scale=alt.Scale(
                scheme='dark2'))).properties(width=300,height=300).interactive().facet('invper1kWp:N',columns=2)

    display(chart)

In [55]:
plot_cumul_costs(df_2023_p_geenT)

alt.FacetChart(...)

In [56]:
def plot_jaarkosten_el(df):
    source = df.reset_index()
    source = source[source['index']<15]
    chart = alt.Chart(source,title='electriciteitskosten per jaar').mark_line(clip=True,width=15).encode(
            x=alt.X('index:N'),
            y=alt.Y('el:Q'),
            color=alt.Color('Wp:N', scale=alt.Scale(
                scheme='category20'))).properties(width=300,height=300).interactive().facet('invper1kWp:N',columns=2)

    display(chart)
    

In [57]:
# 
plot_jaarkosten_el(df_2023_p_geenT)

alt.FacetChart(...)

In [58]:
# source = df.reset_index()
# source = source[(source['index']<10)&(source['index']>3)]
# chart = alt.Chart(source).mark_line(clip=True,width=15).encode(
#         x=alt.X('index:N'),
#         y=alt.Y('total:Q'),
#         color=alt.Color('Wp:N', scale=alt.Scale(
#             scheme='dark2'))).properties(width=300,height=300).interactive().facet('invper1kWp:N',columns=2)
    
# display(chart)

In [59]:
def plottotalekosten(df):
    source = df.reset_index()
    source = source[source['index']<15]
    chart = alt.Chart(source,title='totale jaarlijkse energiekosten').mark_line(clip=True,width=15).encode(
            x=alt.X('index:N'),
            y=alt.Y('gas_el:Q'),
            color=alt.Color('Wp:N', scale=alt.Scale(
            scheme='dark2'))).properties(width=400,height=400).interactive()
    
    display(chart)
    

In [60]:
plottotalekosten(df_2023plafond)

alt.Chart(...)

In [61]:
plottotalekosten(df_2023_p_geenT)

alt.Chart(...)

In [62]:
# source = df.reset_index()
# source = source[source['index']<15]
# chart = alt.Chart(source,title='totale jaarlijkse energiekosten').mark_line(clip=True,width=15).encode(
#         x=alt.X('index:N'),
#         y=alt.Y('gas_el:Q'),
#         color=alt.Color('Wp:N', scale=alt.Scale(
#             scheme='dark2'))).properties(width=400,height=400).interactive()
    
# display(chart)

In [63]:
def plot_besparing(df):
    source = df.reset_index()
    source = source[source['index']<15]
    chart = alt.Chart(source,title='totale jaarlijkse besparing').mark_line(clip=True,width=15).encode(
            x=alt.X('index:N'),
            y=alt.Y('besparing:Q'),
            color=alt.Color('Wp:N', scale=alt.Scale(
                scheme='dark2'))).properties(width=200,height=200).interactive().facet('invper1kWp:N',columns=2)

    display(chart)

In [64]:
plot_besparing(df_2023_p_geenT)

alt.FacetChart(...)

In [65]:
# source = df.reset_index()
# source = source[source['index']<15]
# chart = alt.Chart(source,title='totale jaarlijkse besparing').mark_line(clip=True,width=15).encode(
#         x=alt.X('index:N'),
#         y=alt.Y('besparing:Q'),
#         color=alt.Color('Wp:N', scale=alt.Scale(
#             scheme='dark2'))).properties(width=200,height=200).interactive().facet('invper1kWp:N',columns=2)
    

# display(chart)

In [66]:
# df['rendement'] = df['besparing'] / df['investment']

In [67]:
def plotrendement(df):
    source = df.reset_index()
    source = source[source['index']<15]
    chart = alt.Chart(source,title='totale jaarlijkse besparing').mark_line(clip=True,width=15).encode(
            x=alt.X('index:N'),
            y=alt.Y('rendement:Q'),
            color=alt.Color('Wp:N', scale=alt.Scale(
                scheme='dark2'))).properties(width=300,height=300).interactive().facet('invper1kWp:N',columns=2)


    display(chart)

In [68]:
plotrendement(df_2023_p_geenT)

alt.FacetChart(...)

In [69]:
# source = df.reset_index()
# source = source[source['index']<15]
# chart = alt.Chart(source,title='totale jaarlijkse besparing').mark_line(clip=True,width=15).encode(
#         x=alt.X('index:N'),
#         y=alt.Y('rendement:Q'),
#         color=alt.Color('Wp:N', scale=alt.Scale(
#             scheme='dark2'))).properties(width=300,height=300).interactive().facet('invper1kWp:N',columns=2)
    

# display(chart)

In [70]:
df = pd.concat([df_oud,df_huidig,df_2023geenplafond,df_2023plafond,df_2023_gp_geenT,df_2023_p_geenT])

In [71]:
df = df.reset_index()
df.rename(columns={"index": "yr"},inplace=True)
df.head()

,yr,EH,EL,Gas_VW,Gas_SWW,E_terug_H,E_terug_L,E_eigen_H,E_eigen_L,gas,...,gas_el,gas_el_cumul,investment,total,invper1kWp,Wp,selectedbeleid,selectedprijs,besparing,rendement
0,0,481.678795,172.576933,1052.56026,294.61764,0.0,0.0,0.0,0.0,1347.1779,...,2001.433628,2001.433628,0.0,2001.433628,1.0,0,1_2022_orig,2022_1,0.0,NaN
1,1,481.678795,172.576933,1052.56026,294.61764,0.0,0.0,0.0,0.0,1347.1779,...,2001.433628,4002.867256,0.0,4002.867256,1.0,0,1_2022_orig,2022_1,0.0,NaN
2,2,481.678795,172.576933,1052.56026,294.61764,0.0,0.0,0.0,0.0,1347.1779,...,2001.433628,6004.300883,0.0,6004.300883,1.0,0,1_2022_orig,2022_1,0.0,NaN
3,3,481.678795,172.576933,1052.56026,294.61764,0.0,0.0,0.0,0.0,1347.1779,...,2001.433628,8005.734511,0.0,8005.734511,1.0,0,1_2022_orig,2022_1,0.0,NaN
4,4,481.678795,172.576933,1052.56026,294.61764,0.0,0.0,0.0,0.0,1347.1779,...,2001.433628,10007.168139,0.0,10007.168139,1.0,0,1_2022_orig,2022_1,0.0,NaN


In [72]:
df['scenario'] = df['selectedbeleid'] + '_' + df['selectedprijs']

In [73]:
df[(df['yr']==20)&(df['invper1kWp']==1.5)]

,yr,EH,EL,Gas_VW,Gas_SWW,E_terug_H,E_terug_L,E_eigen_H,E_eigen_L,gas,...,gas_el_cumul,investment,total,invper1kWp,Wp,selectedbeleid,selectedprijs,besparing,rendement,scenario
470,20,481.678795,172.576933,1052.56026,294.61764,0.000000,0.000000,0.000000,0.000000,1347.17790,...,42030.106183,0.0,42030.106183,1.5,0,1_2022_orig,2022_1,0.000000,NaN,1_2022_orig_2022_1
495,20,365.227995,127.456277,1052.56026,294.61764,7.969671,3.432622,116.450801,45.120656,1347.17790,...,38319.232677,1500.0,39819.232677,1.5,1000,1_2022_orig,2022_1,172.973749,0.115316,1_2022_orig_2022_1
520,20,308.348041,105.733045,1052.56026,294.61764,39.161921,18.507742,173.330754,66.843887,1347.17790,...,35381.858508,3000.0,38381.858508,1.5,2000,1_2022_orig,2022_1,297.844305,0.099281,1_2022_orig_2022_1
545,20,276.567099,93.800045,1052.56026,294.61764,80.138552,38.454456,205.111697,78.776888,1347.17790,...,32769.835299,4500.0,37269.835299,1.5,3000,1_2022_orig,2022_1,402.481593,0.089440,1_2022_orig_2022_1
570,20,255.589434,85.960472,1052.56026,294.61764,125.326639,60.438050,226.089361,86.616461,1347.17790,...,30296.877884,6000.0,36296.877884,1.5,4000,1_2022_orig,2022_1,498.470511,0.083078,1_2022_orig_2022_1
595,20,240.286357,80.265768,1052.56026,294.61764,172.726858,83.488924,241.392438,92.311165,1347.17790,...,28181.004671,7500.0,35681.004671,1.5,5000,1_2022_orig,2022_1,589.919384,0.078656,1_2022_orig_2022_1
620,20,228.425815,75.848931,1052.56026,294.61764,221.469084,107.175662,253.252980,96.728001,1347.17790,...,26313.659154,9000.0,35313.659154,1.5,6000,1_2022_orig,2022_1,678.625728,0.075403,1_2022_orig_2022_1
645,20,218.898302,72.276814,1052.56026,294.61764,271.120798,131.282730,262.780493,100.300119,1347.17790,...,24527.064004,10500.0,35027.064004,1.5,7000,1_2022_orig,2022_1,765.484140,0.072903,1_2022_orig_2022_1
670,20,211.014063,69.277942,1052.56026,294.61764,321.413112,155.675044,270.664732,103.298991,1347.17790,...,22761.222899,12000.0,34761.222899,1.5,8000,1_2022_orig,2022_1,851.051878,0.070921,1_2022_orig_2022_1
1820,20,822.638765,282.823017,1530.53338,473.85756,0.000000,0.000000,0.000000,0.000000,2004.39094,...,65306.907158,0.0,65306.907158,1.5,0,2_2022_aangepast,2022_2,0.000000,NaN,2_2022_aangepast_2022_2


In [74]:
def plottotalekostennaastelkaar(df,yr,kwp):
    source = df[(df['yr']==yr)&(df['invper1kWp']==kwp)].reset_index()
# source = source[(source['index']<10)&(source['index']>3)]
    chart = alt.Chart(source,title='Totale kosten in 20 yr incl investering').mark_bar(clip=True,width=15).encode(
        x=alt.X('Wp:N'),
        y=alt.Y('total:Q'),
        color=alt.Color('scenario:N', scale=alt.Scale(
            scheme='dark2'))).properties(width=200,height=200).interactive().facet('scenario:N',columns=3)
    
    display(chart)

In [75]:
# periode van aantal jaren tegen aanschafprijs van X per Wp
plottotalekostennaastelkaar(df,20,1.75)

alt.FacetChart(...)

In [76]:
from shapely.geometry import LineString

In [77]:
def berekensnijpunt(reflijn,calclijn):
    
    '''input reflijn, anderelijn'''
    
    try:   
        
        lijn1 = LineString(np.column_stack((reflijn['yr'],reflijn['total'])))
        
        lijn2 = LineString(np.column_stack((calclijn['yr'],calclijn['total'])))
        intersection = lijn1.intersection(lijn2)
        snijpunt = intersection.x
    except: 
            snijpunt = 100
            print('geen snijpunt')
    
    
    return snijpunt

In [78]:
scenarios = df['scenario'].unique().squeeze()
# [s for s in df['scenario'].unique().squeeze()]

In [79]:
def berekensnijpunten(df):
    # result = pd.DataFrame({})
    
    result = ([])
    ref= df[(df['invper1kWp']==1)&(df['Wp']==0)]
    scen = df['scenario'].unique().squeeze()
    
    for p in prijsrange:
        for wp in E_gem_jaar.index[1:]:
            lijn2_df = df[(df['invper1kWp']==p)&(df['Wp']==wp)]
            sp = berekensnijpunt(ref,lijn2_df)
            result.append([sp,p,wp])
            # result.append(sp,ignore_index=False)
            # print (round(sp,2))
    result=pd.DataFrame(result)
    return result

In [80]:
df_terugjr = pd.DataFrame()
for s in scenarios:
    a = berekensnijpunten(df[df['scenario']==s])
    a['scenario']= s 
    df_terugjr = df_terugjr.append(a,ignore_index=False)
    


In [81]:
a

,0,1,2,scenario
0,1.994589,1.00,1000,3_2023_2023_p_gT
1,2.085220,1.00,2000,3_2023_2023_p_gT
2,2.149490,1.00,3000,3_2023_2023_p_gT
3,2.195220,1.00,4000,3_2023_2023_p_gT
4,2.595813,1.00,5000,3_2023_2023_p_gT
5,3.301775,1.00,6000,3_2023_2023_p_gT
6,4.156167,1.00,7000,3_2023_2023_p_gT
7,5.068364,1.00,8000,3_2023_2023_p_gT
8,2.770549,1.25,1000,3_2023_2023_p_gT
9,2.931889,1.25,2000,3_2023_2023_p_gT


In [82]:
df_terugjr.rename({0:'jaren',1:'gemprijs',2:'Wp'},inplace=True,axis=1)
df_terugjr

,jaren,gemprijs,Wp,scenario
0,4.411590,1.00,1000,1_2022_orig_2022_1
1,4.613875,1.00,2000,1_2022_orig_2022_1
2,4.755078,1.00,3000,1_2022_orig_2022_1
3,4.854486,1.00,4000,1_2022_orig_2022_1
4,5.275346,1.00,5000,1_2022_orig_2022_1
...,...,...,...,...
43,8.030964,2.25,4000,3_2023_2023_p_gT
44,10.031879,2.25,5000,3_2023_2023_p_gT
45,12.977251,2.25,6000,3_2023_2023_p_gT
46,16.095421,2.25,7000,3_2023_2023_p_gT


In [83]:
# dummyref = df[(df['invperm2']==300)&(df['m2']==0)]
# lijn2 = df[(df['invperm2']==400)&(df['m2']==40)]

# lijn1 = LineString(np.column_stack((dummyref['yr'],dummyref['total'])))
# lijn2 = LineString(np.column_stack((lijn2['yr'],lijn2['total'])))
# intersection = lijn1.intersection(lijn2)

# berekensnijpunt(dummyref,lijn2)

In [84]:

# ref_df= df[(df['invperm2']==300)&(df['m2']==0)]
# for p in prijsrange:
#     for opp in E_gem_jaar.index[1:]:
    

#         lijn2_df = df[(df['invperm2']==p)&(df['m2']==opp)]
#         sp = berekensnijpunt(ref_df,lijn2_df)
#         print (round(sp,2))
#     print('')


In [85]:
df_terugjr['scenario'].unique()

array(['1_2022_orig_2022_1', '2_2022_aangepast_2022_2',
       '3_2023_2023_geenplafond', '3_2023_2023_plafond',
       '3_2023_2023_gp_gT', '3_2023_2023_p_gT'], dtype=object)

In [86]:
def plotterugverdientijd(df):
    source = df#.reset_index()
# source = source[source['index']<15]
    chart = alt.Chart(source,title='terugverdientijd').mark_point(clip=True,width=15).encode(
        x=alt.X('gemprijs:Q',title='Installatieprijs Euro/Wp'),
        y=alt.Y('jaren:Q',scale=alt.Scale(domain=[0, 20])),
        color=alt.Color('Wp:N', scale=alt.Scale(
            scheme='dark2'))).properties(width=200,height=250).interactive().facet('scenario:N',columns=3)
    
    

    display(chart)

In [87]:
plotterugverdientijd(df_terugjr)

alt.FacetChart(...)

prijsscenario : huidige prijs, oud beleid, 0.09 teruglevering


# NOOT: 
DE TERUG LEVER VERGOEDING bepaalt de paybacktime

In [88]:
# calculatie met hand
stel installatie 1.5 euro/Wp


SyntaxError: invalid syntax (2777977252.py, line 2)

In [ ]:
E_gem_jaar[colsforcalc]

In [ ]:
source = df_terugjr.reset_index()
# source = source[source['index']<15]
chart = alt.Chart(source,title='terugverdientijd').mark_point(clip=True,width=15).encode(
        x=alt.X('gemprijs:Q'),
        y=alt.Y('jaren:Q'),
        color=alt.Color('Wp:N', scale=alt.Scale(
            scheme='dark2'))).properties(width=400,height=400).interactive()
    

display(chart)

Wanneer we de kostprijs gaan ontleden per onderdeel, komen we uit op het volgende voor een gemiddelde offerte:

    Zonnepanelen: 35%
    Werkuren en keuring: 30%
    Omvormer: 20%
    Montagemateriaal: 10%
    Kabels en kleine onderdelen: 5%
    
    https://zonnepanelenenergie.be/prijzen
    

In [ ]:
prijzen_input

In [ ]:
df

HUIDIGE CONCLUSIE: NIET MEER DAN 25m2 voor huidige gebruik.

25m2 ~ ongeveer 16 panelen van 1.6m2 

25m2 * 300 euro --> 7500 k Euro.



Vragen wat als electrische auto / electrische boiler/ warmtepomp
warmtepomp in combinatie met wegvallen van saldering
en what about een accu?


Doel:   
bouw functie die input verschillende prijzen + verschillen beleid als input,  
dan berekening afh aantal m2  



https://volt-zonnepanelen.nl/product/10-zonnepanelen-set-3250-wp-inclusief-installatie-of-als-doe-het-zelf-pakket-met-omvormer-en-montagemateriaal/

voor 3900 Wp, kosten 6.5 k Ex btw --> 1.67 euro/Wp
10 panelen a 1.92 m2 = 19.20 m2 --> 338 euro/m2 



In [ ]:
def bereken_metrics(prijs,beleid,
                    
                    df = pd.DataFrame()
for p in prijsrange:
    for Wp in E_gem_jaar.index:
        a1 = berekenjaren(selectedprijs,E_gem_jaar.loc[Wp],Gas,selectedbeleid,s_dummy,p,Wp)
        df = df.append(a1,ignore_index=False)

# oude berekening

In [35]:
# # prijzen per kWh(EH +EL), m3 (G), dag (Netbeheer) en jaar (vast)

# prijzen = {'EH':0.245 ,'EL':0.195,'G':1.0765,
#            'net_EH':267/365/2,'net_EL':267/365/2,'net_G':186.55,
#            'vast_EH':80/2,'vast_EL':80/2,'vast_G':80,
#            'IN_EX':'IN','BTW':0.21,
#           'TH':0.09, 'TL':0.09 ,'refyr':2022}
# prijzen = pd.DataFrame(prijzen,index=['2022_1'])
# prijzen = prijzen.transpose()
# prijzen
                                      

In [36]:
# def calc_prices_ex(columnX,beleid):
#     """ bereken de prijzen die in de berekening moeten
#     als input al incl belastingen is 
#     als input exl belastingen ingegeven is,neem dan de getallen gewoon over
    
#     """
#     # print(columnX)

#     if columnX['IN_EX']=='IN':
#         EH = columnX['EH']/(1+beleid['BTW_EH']) - beleid['ODE_EH'] - beleid['EB_EH']
#         EL = columnX['EL']/(1+beleid['BTW_EL']) - beleid['ODE_EL'] - beleid['EB_EL']
#         G = columnX['G']/(1+beleid['BTW_G']) - beleid['ODE_G'] - beleid['EB_G']
#         net_EH = columnX['net_EH']/(1+beleid['BTW_EH'])
#         net_EL = columnX['net_EL']/(1+beleid['BTW_EL'])
#         net_G = columnX['net_G']/(1+beleid['BTW_G'])
#         vast_EH = columnX['vast_EH']/(1+beleid['BTW_EH'])
#         vast_EL = columnX['vast_EL']/(1+beleid['BTW_EL'])
#         vast_G = columnX['vast_G']/(1+beleid['BTW_G'])
#         TH = columnX['TH']/(1+beleid['BTW_EH'])
#         TL = columnX['TL']/(1+beleid['BTW_EL'])
        
        
#     else:
#         print('input is excl belastingen')
#         EH = columnX['EH']
#         EL = columnX['EL']
#         G = columnX['G']
#         net_EH = columnX['net_EH']
#         net_EL = columnX['net_EL']
#         net_G = columnX['net_G']
#         vast_EH = columnX['vast_EH']
#         vast_EL = columnX['vast_EL']
#         vast_G = columnX['vast_G']
#         TH = columnX['TH']
#         TL = columnX['TL']
        
        
        
        
#     return {'EH':EH,'EL':EL,'G':G,
#            'net_EH' :net_EH, 'net_EL' : net_EL , 'net_G' : net_G,
#            'vast_EH' : vast_EH , 'vast_EL' : vast_EL ,'vast_G':vast_G,
#            'TH':TH , 'TL':TL}

In [37]:
# # dit zou getallen moeten opleveren gelijk aan 2 kolom in prijzen[_exbtw]
# prijsnu = calc_prices_ex(prijzen['2022'],beleid['2022_orig'])
# prijsnu

In [38]:
def vermenigvuldig(prijs,beleid,verbruik,keuze):
    """ vermenigvuldig de prijs 
    """
    var = ['ODE','EB','net','vast','BTW','BT']
    var = [''.join([s,'_',keuze]) for s in var]
    
    
    
    if keuze == 'G':
        v = verbruik['G_SWW'] + verbruik['G_VW']
    else:
        v = verbruik.get(keuze)
    
    p = prijs.get(keuze) 
    p = (p + beleid[var[0]]+beleid[var[1]]) * v +\
        prijs[var[2]]*365 + prijs[var[3]] - beleid[var[5]]
    p = p*(1+beleid[var[4]])
    
    return p

    

In [21]:
def kosten(prijs,beleid,verbruik):
    """
    input: prijs excl taxes , beleid, verbruik
    
    return kosten incl taxes:
    kosten electriciteit per jaar(1), kosten electriciteit per maand (2), kosten electriciteit per unit(3)
    kosten gas per jaar (4) , kosten gas per maand (5), kosten gas per unit (6)
    kosten gas+electriciteit per jaar(7), kosten gas+electriciteit per maand (8)
    """
    EH = vermenigvuldig(prijs,beleid,verbruik,'EH')
    EL = vermenigvuldig(prijs,beleid,verbruik,'EL')
    
    # E = EH + EL 
    # E_unit = E/(verbruik['EH']+verbruik['EL'])
    
    G_SWW = vermenigvuldig(prijs,beleid,verbruik,'G_SWW')
   
    G = vermenigvuldig(prijs,beleid,verbruik,'G')
    G_unit = G/(verbruik['G_SWW']+verbruik['G_VW'])
    
    EandG = [E,E/12,E_unit,G,G/12,G_unit,E+G, (E+G)/12]
    EandG = [round(nr,2) for nr in EandG]
    
    # format: Electriciteit Jaar, Electriciteit Maand, Electriciteit Unit,
    #         Gas jaar, Gas maand, Gas unit
    #         E+G jaar, E+G maand
    
    return EandG

In [22]:
# bereken de kosten met geselecteerde prijs, geselecteerd beleid en een gekozen verbruik
kosten(prijsnu,beleid['2022_orig'],verbruik['2022'])

[566.06, 47.17, 0.14, 1795.18, 149.6, 1.26, 2361.24, 196.77]

In [14]:
kosten(prijsnu,beleid['2022_aangepast'],verbruik['2022'])

[271.2, 22.6, 0.07, 1617.15, 134.76, 1.14, 1888.35, 157.36]

In [15]:
verbruik['scen1'] = [5000,1600,240,400]

In [16]:
kosten(prijsnu,beleid['2022_aangepast'],verbruik['scen1'])

[845.03, 70.42, 0.13, 860.75, 71.73, 1.34, 1705.78, 142.15]

In [17]:
kosten(prijzen['2022_ex'],beleid['2022_aangepast'],verbruik['2022'])

[800.62, 66.72, 0.2, 2652.82, 221.07, 1.87, 3453.44, 287.79]

In [18]:
prijzen

,2022,2022_ex
EH,0.245,0.13737
EL,0.195,0.09718
G,1.0765,0.44001
net_EH,0.3667,0.30305
net_EL,0.3667,0.30305
net_G,0.5111,0.4224
vast_EH,40.0,33.06
vast_EL,40.0,33.06
vast_G,80,66.12
IN_EX,IN,EX


In [19]:
prijzen['2022gaslicht'] = prijzen['2022']
prijzen.loc['EH','2022gaslicht'] = 0.37
prijzen.loc['EL','2022gaslicht'] = 0.32
prijzen.loc['G','2022gaslicht'] = 1.42

In [20]:
prijsonline = calc_prices_ex(prijzen['2022gaslicht'],beleid['2022_orig'])
kosten(prijsonline,beleid['2022_aangepast'],verbruik['2022'])

[721.62, 60.13, 0.18, 2056.54, 171.38, 1.45, 2778.16, 231.51]

In [21]:
prijsonline

{'EH': 0.23849512396694217,
 'EL': 0.19717280991735536,
 'G': 0.7238337190082643,
 'net_EH': 0.30305785123966944,
 'net_EL': 0.30305785123966944,
 'net_G': 0.422396694214876,
 'vast_EH': 33.057851239669425,
 'vast_EL': 33.057851239669425,
 'vast_G': 66.11570247933885}

In [22]:
prijssel = calc_prices_ex(prijzen['2022gaslicht'],beleid['2022_orig']) #prijzen gedefinieerd tegen huidig btw pct
kosten(prijssel,beleid['2022_aangepast'],verbruik['scen1'])

[1588.21, 132.35, 0.24, 1058.79, 88.23, 1.65, 2647.0, 220.58]

In [23]:
verbruik['scen2'] = [5200,1600,1,1]
kosten(prijssel,beleid['2022_aangepast'],verbruik['scen2'])

[1654.87, 137.91, 0.24, 242.67, 20.22, 121.34, 1897.55, 158.13]

In [24]:
verbruik['scen3'] = [1000,600,1,1]
kosten(prijssel,beleid['2022_orig'],verbruik['scen3'])

[228.06, 19.01, 0.14, 269.39, 22.45, 134.7, 497.45, 41.45]

In [25]:
150*12

1800

In [26]:
hybride=6000
zonnepanelen = 25 * 300 # 25*250 = 7500

In [27]:
hybride+ zonnepanelen

13500

In [28]:
verbruik

,2022,scen1,scen2,scen3
EH,2400,5000,5200,1000
EL,1600,1600,1600,600
G_SWW,240,240,1,1
G_VW,1180,400,1,1


In [ ]:
doel: 
    eerst bereken gevolgen voor een jaar, 
    dus inputs:
    1.prijs
    2.beleid
    3.onttrekking net (H+L), terug levering (H+L) , eigen_gebruik(voor QC) (H+L) = scenario met meerdere m2.
output voor verschillende m2, 
        de netto jaarkosten E+G, 
        EL, jaarkosten,  
        EH, jaarkosten,
        TL, jaarteruglevering,
        TH, jaarteruglevering
        G_SWW, jaarkosten
        G_VW, jaarkosten

        show maandgetallen
        show unitgetallen
        
vergelijk scenario (ref_scenario, active scenario)
        verschil tussen 2 scenario alle variabelen voor 
        
maak vervolgens tijdserie waarbij 
    1. scenario per jaar
    2. keuze prijs per jaar, inclusief aanname inflatie
    3. 

In [25]:
E_gem_jaar =  pd.read_pickle(".//E_gem_perjaar.pkl") 
E_gem_jaar
# E_gem_jaar.to_pickle('./E_gem_perjaar.pkl')

,yr,Elektriciteit_cumul,opbrengst_cumul,gebruik_eigen_cumul,gebruik_net_cumul,terugleveren_cumul,zon_laag_kWh_cumul,zon_hoog_kWh_cumul,eigengebruik_laag_kWh_cumul,eigengebruik_hoog_kWh_cumul,gebruiknet_laag_kWh_cumul,gebruiknet_hoog_kWh_cumul,teruglever_laag_kWh_cumul,teruglever_hoog_kWh_cumul
m2,,,,,,,,,,,,,,
0,2016.0,4133.156545,0.000000,0.000000,4133.156545,0.000000,0.000000,0.000000,0.000000,0.000000,1552.472818,2580.683727,0.000000,0.000000
5,2016.0,4133.156545,790.921970,699.611735,3433.544811,91.310235,258.328788,532.593182,229.408742,470.202992,1323.064076,2110.480735,28.920045,62.390189
10,2016.0,4133.156545,1581.843939,1069.586803,3063.569742,512.257136,516.657576,1065.186364,348.980606,720.606197,1203.492212,1860.077530,167.676970,344.580167
15,2016.0,4133.156545,2372.765909,1281.496318,2851.660227,1091.269591,774.986364,1597.779545,417.362523,864.133795,1135.110295,1716.549932,357.623841,733.645750
20,2016.0,4133.156545,3163.687879,1422.174485,2710.982061,1741.513394,1033.315152,2130.372727,463.295485,958.879000,1089.177333,1621.804727,570.019667,1171.493727
25,2016.0,4133.156545,3954.609848,1524.461356,2608.695189,2430.148492,1291.643939,2662.965909,496.358848,1028.102508,1056.113970,1552.581220,795.285091,1634.863402
30,2016.0,4133.156545,4745.531818,1603.779273,2529.377273,3141.752545,1549.972727,3195.559091,522.120318,1081.658955,1030.352500,1499.024773,1027.852409,2113.900136
35,2016.0,4133.156545,5536.453788,1667.897053,2465.259492,3868.556735,1808.301515,3728.152273,543.187674,1124.709379,1009.285144,1455.974348,1265.113841,2603.442894
40,2016.0,4133.156545,6327.375758,1720.787152,2412.369394,4606.588606,2066.630303,4260.745455,560.493152,1160.294000,991.979667,1420.389727,1506.137152,3100.451455


# Zonne energie.

In [29]:
import os 
import io
import types
import knmi_gn_001 as gethdd

importing Jupyter notebook from knmi_gn_001.ipynb


In [7]:
knmi = gethdd.retrieveHDD()

read the csv file with knmi data  knmi_data.csv
last data point in csv file:  2022-06-05 00:00:00
The current time is : 2022-06-06 21:51:34.793736
The number of days since last datapoint is : 1
KNMI last datapoint is : 2022-06-05 00:00:00
Current time : 2022-06-06 21:51:34.801409
Elapsed time since  [hours] :  45
Nr rows to add :  1
               SQ       Q    DR     TG    FG     RH        HDD       dow
datetime                                                                
2022-06-02  139.0  2739.0   0.0  133.0  15.0    0.0   1.700000  Thursday
2022-06-03  145.0  2911.0   0.0  164.0  37.0    0.0   0.066667    Friday
2022-06-04  130.0  2663.0   0.0  162.0  37.0    0.0   0.266667  Saturday
2022-06-05   16.0  1072.0  72.0  168.0  37.0  211.0   0.000000    Sunday
2022-06-06    0.0     0.0   0.0    0.0   0.0    0.0  14.000000    Monday


In [18]:
import pandas as pd
import datetime 

Q         = Globale straling (in J/cm2) / Global radiation (in J/cm2)

In [56]:
annuals = knmi[datetime.datetime(2010,1,1):datetime.datetime(2021, 12, 31)].resample('Y').sum()
annuals

,SQ,Q,DR,TG,FG,RH,HDD
datetime,,,,,,,
2010-12-31,17106.0,374371.0,7336.0,31411.0,10351.0,8016.0,2890.666667
2011-12-31,17629.0,370025.0,6768.0,37852.0,11403.0,8688.0,2241.600000
2012-12-31,16263.0,361882.0,6007.0,36275.0,11038.0,7576.0,2445.533333
2013-12-31,15841.0,362465.0,5744.0,34467.0,11228.0,7727.0,2635.900000
2014-12-31,17275.0,372039.0,5301.0,41238.0,10496.0,7291.0,1929.633333
2015-12-31,18045.0,378964.0,7316.0,38078.0,11611.0,8303.0,2272.300000
2016-12-31,17844.0,377376.0,5997.0,37191.0,10373.0,6654.0,2363.433333
2017-12-31,16944.0,368559.0,6899.0,38325.0,10812.0,8100.0,2228.933333
2018-12-31,21076.0,419013.0,4815.0,40140.0,10750.0,5362.0,2231.733333


In [57]:
annuals['Q'].mean()

379203.25

Per paneel ca 250 kWh  
van 1.65 m2   
250 000 Wh per 165 cm2 --> 15.15 Wh per cm2 per jaar

In [66]:
paneel = 260 # kWh

In [67]:
paneel*1e3/16500 # Wh / cm2

15.757575757575758

In [68]:
paneel_Joule = paneel * 1e3 / 16500 * 3600  # Joule / cm2
paneel_Joule

56727.27272727273

In [69]:
paneel_Joule /annuals['Q'].mean() 

0.14959595606649662

In [70]:
solar_rend = paneel_Joule /annuals['Q'].mean() 
solar_rend

0.14959595606649662

In [84]:
knmi['paneel']=knmi['Q']* solar_rend * 16500 /3600 / 1e3

In [93]:
solar_maand = knmi[datetime.datetime(2010,1,1):datetime.datetime(2021, 12, 31)]['paneel'].resample('M').sum().reset_index()

In [95]:
solar_maand

,datetime,paneel
0,2010-01-31,4.951751
1,2010-02-28,6.914076
2,2010-03-31,19.263284
3,2010-04-30,33.286846
4,2010-05-31,33.633098
...,...,...
139,2021-08-31,30.743777
140,2021-09-30,22.553024
141,2021-10-31,13.031428
142,2021-11-30,5.833494


In [87]:
import altair as alt

In [8]:
def plot_paneel(df):
    source = df
    chart2a = alt.Chart(df,title='een paneel doet per maand').mark_bar(
            strokeWidth=1).encode(
            x=alt.X('datetime:T',
                axis=alt.Axis(title="tijd")),
            y=alt.Y("paneel:Q",
                axis=alt.Axis(title="solar per paneel [kWh]")),
        color=alt.Color('dy:N')
        ).properties(width=600,height=450).interactive()
    display(chart2a)



# def plotdata(df):
#     wdth = 300
#     source = df
#     chart2a = alt.Chart(df[df['variable']!='gas'],
#                                title='elektriciteit cumulatief').mark_line(
#             strokeWidth=1).encode(
#             x=alt.X('normalday:T',
#                 axis=alt.Axis(title="tijd")),
#             y=alt.Y("cumul:Q",
#                 axis=alt.Axis(title="verbruik [kWh]")),
#         color=alt.Color('dy:N')
#         ).properties(width=wdth,height=250).interactive()


#     chart2b = alt.Chart(df[df['variable']=='gas'],
#                         title='gas cumulatief').mark_line(
#             strokeWidth=1).encode(
#             x=alt.X('normalday:T'),
#             y=alt.Y("cumul:Q",
#                 axis=alt.Axis(title="verbruik [m3]")),
#         color=alt.Color('dy:N')
#         ).properties(width=wdth,height=250).interactive()

#     # chart_all=alt.vconcat(chart2a|chart2b)
#     # display(chart_all)

In [ ]:
plot_paneel(solar_maand)

In [102]:
solar_maand.set_index('datetime')

,paneel
datetime,
2010-01-31,4.951751
2010-02-28,6.914076
2010-03-31,19.263284
2010-04-30,33.286846
2010-05-31,33.633098
...,...
2021-08-31,30.743777
2021-09-30,22.553024
2021-10-31,13.031428


In [105]:
solar_maand.dtypes

datetime    datetime64[ns]
paneel             float64
dtype: object

In [107]:
solar_maand['paneel'].mean()*12

260.00000000000006

In [109]:
360 / 3

120.0

In [110]:
120*45

5400

In [111]:
360/7

51.42857142857143

In [112]:
360/20

18.0

In [113]:
22 / 30 

0.7333333333333333

In [114]:
12 / 0.73 # aantal panelen nodig om 12 kWh per dag te kunnen leveren (gemiddeld)

16.438356164383563

In [115]:
16 * 45 / 30 # per dag genereren in de zomer

24.0

In [63]:
3*2+1*3

9

In [65]:
9*60 

540